#Baixando o Spark

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

!rm -rf spark-2.4.4-bin-hadoop2.7.tgz

#Baixando o java 8

In [ ]:
!apt update 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

#Importando variáveis de ambiente

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

#Importando funções

In [ ]:
def limpeza(text): #Criando estrutura de dados para o arquivo da amazon
  text = text.split('\r\n')
  flag = 0
  vet = {}
  for index,words in enumerate(text):

    if words.startswith('Id:'):
      vet['id'] = words.split()[1].strip()

    elif words.startswith('ASIN:'):
      vet['asin'] = words.split('ASIN:')[1].strip()
        
    elif words.startswith('  title:'):
      vet['title'] = words.split('title: ')[1].strip()

    elif words.startswith('  group:'):
      vet['group'] = words.split('group:')[1].strip()

    elif words.startswith('  salesrank:'):
      vet['salesrank'] = words.split('salesrank:')[1].strip()

    elif words.startswith('  similar:'):
      if words.strip()[9] != 0:
        vet['similar'] = words.split()[2:]
      
    elif words.startswith('  categories:'):
      vet['categories'] = []

    elif words.startswith('   |'):
      categories = words.split('|')
      if len(categories) == 1:
          pass
      else:
          vet['categories'].append(categories[len(categories)-1])

    elif words.startswith('  reviews:'):
      vet['avg_rating'] =  words.split(': ')[4]
      vet['reviews'] = []

    elif words.startswith('    2') or words.startswith('    1'):
      aux = words.split()
      vet['reviews'].append({'data':aux[0], 'cliente':aux[2], 
                             'rating':aux[4], 'votes':aux[6], 'helpful':aux[8]})
            
  return vet

def limpeza2(word): #remove o id[0]
  if len(word) != 0 and len(word) > 2 :
    return word

def transformList(word): #transformar para lista para fazer melhor manipulações
  lista = []
  for i in word:
    for j in i:
      lista.append(j)

  return lista

def media(anterior, valor, tamanho): #media de diaas
  vet = float(anterior) + (float(valor)-float(anterior))/float(tamanho)
  return round(vet,5)

def oi2(busca): #Seleciona dados com base group == group e media in lista_media[]
  global filtrado
  contador = 0
  vet = {}
  vet['result'] = []
  for i in filtrado:
    if i[0] == busca[1] and busca[2] in i[1]:
      vet['result'].append({'title': busca[0], 'group': busca[1], 'avg':busca[2]}) 
      contador+=1

  return vet



#Importanto biblioteca e Escolhendo o Produto


In [ ]:
from pyspark import SparkConf, SparkContext, SparkFiles
import numpy as np
import math
import operator
import statistics
import re

ID_PRODUTO = '2'

conf = SparkConf().setMaster("local[*]").setAppName("Spark")
sc = SparkContext.getOrCreate(conf=conf)
sc._jsc.hadoopConfiguration().set("textinputformat.record.delimiter", "\r\n\r\n")

#Colocando o arquivo amazon-meta.txt no RDD

In [ ]:
url = "https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz"

sc.addFile(url)
file_name = re.split(pattern='/', string=url)[-1]
file_name = SparkFiles.get(file_name)
text_file = sc.textFile(file_name)

#Testando o arquivo no RDD

In [ ]:
print(text_file.take(2))

['# Full information about Amazon Share the Love products\r\nTotal items: 548552', 'Id:   0\r\nASIN: 0771044445\r\n  discontinued product']


#1) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação


In [ ]:
lines = text_file.map(limpeza).filter(limpeza2)

filtrado = lines.filter(lambda x: x['id'] == ID_PRODUTO).map(lambda x: x['reviews']).take(1)
comentarios = transformList(filtrado)

comentarios_uteis = sorted(comentarios, key = lambda i: (int(i['helpful']), int(i['rating'])), reverse=True)[0:5]
comentarios_uteis = sorted(comentarios_uteis, key = lambda i: int(i['rating']), reverse=True)

comentarios_menos_uteis = sorted(comentarios, key = lambda i: (int(i['helpful']), int(i['rating'])))[0:5]
comentarios_menos_uteis = sorted(comentarios_uteis, key = lambda i: int(i['rating']))


print("5 comentários mais úteis ordenados pela maior avaliação")
for i in range(0,5):
 print(comentarios_uteis[i])


print("\n5 comentários mais úteis ordenados pela menor avaliação")
for i in range(0,5):
 print(comentarios_menos_uteis[i])

5 comentários mais úteis ordenados pela maior avaliação
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-5-23', 'cliente': 'A1GIL64QK68WKL', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'}
{'data': '2004-2-11', 'cliente': 'A1CP26N8RHYVVO', 'rating': '1', 'votes': '13', 'helpful': '9'}

5 comentários mais úteis ordenados pela menor avaliação
{'data': '2004-2-11', 'cliente': 'A1CP26N8RHYVVO', 'rating': '1', 'votes': '13', 'helpful': '9'}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'}
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-5-23

#2) Dado um produto, listar os produtos similares com maiores vendas do que ele


In [ ]:
filtrado = lines.filter(lambda x: x['id'] == ID_PRODUTO).map(lambda x: [x['similar'],x['salesrank']]).take(1)
result = lines.filter(lambda x: x['asin'] in filtrado[0][0] and int(x['salesrank']) < int(filtrado[0][1])).map(lambda x: [x['title'], x['salesrank']]).collect()

print("Dado um produto, listar os produtos similares com maiores vendas do que ele são:")
for i in result:
  print(i)

Dado um produto, listar os produtos similares com maiores vendas do que ele são:
['Yule: A Celebration of Light and Warmth', '103012']
['Midsummer: Magical Celebrations of the Summer Solstice', '159277']
['Lammas', '58836']


#3) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

In [ ]:
filtrado = lines.filter(lambda x: x['id'] == ID_PRODUTO).map(lambda x: x['reviews']).collect()
filtrado = transformList(filtrado)

vet_aux = [0]
for word in filtrado:
    vet_aux.append(media( vet_aux[len(vet_aux)-1], float(word['rating']), len(vet_aux))) 
vet_aux.pop(0)


print('A evolução diária das médias de avaliação ao longo do intervalo de tempo: ')
contador = 0
for word in filtrado:
  print('data:',word['data'],' rating:',word['rating'], ' average:',end='')
  print(vet_aux[contador])
  contador+=1


A evolução diária das médias de avaliação ao longo do intervalo de tempo: 
data: 2001-12-16  rating: 5  average:5.0
data: 2002-1-7  rating: 4  average:4.5
data: 2002-1-24  rating: 5  average:4.66667
data: 2002-1-28  rating: 5  average:4.75
data: 2002-2-6  rating: 4  average:4.6
data: 2002-2-14  rating: 4  average:4.5
data: 2002-3-23  rating: 4  average:4.42857
data: 2002-5-23  rating: 5  average:4.5
data: 2003-2-25  rating: 5  average:4.55556
data: 2003-11-25  rating: 5  average:4.6
data: 2004-2-11  rating: 1  average:4.27273
data: 2005-2-7  rating: 5  average:4.33334


#4) Listar os 10 produtos líderes de venda em categoria

In [ ]:
categoria = "Criminology[11005]"

def oi(word):
  global categoria
  vet = []
  for i in word['categories']:
    if categoria == i:
      vet.append((word['title'],i,int(word['salesrank']),int(word['id'])))
  return vet

filtrado = lines.map(oi).flatMap(lambda x: x).sortBy(lambda x: (x[1],-x[2])).take(10)

print('Categoria:',filtrado[0][1])
for i in filtrado:
  print('  ','ID',i[3],'',end='')
  print(i[0],'---- SalesRank:',i[2])
  contador+=1



Categoria: Criminology[11005]
   ID 196343 Uniform Crime Reports: Crime in the U.S., 1998 (Uniform Crime Reports: Crime in the U.S.) ---- SalesRank: 3158228
   ID 30755 Searching for a Demon: The Media Construction of the Militia Movement ---- SalesRank: 3092830
   ID 431227 Violent Racism: Victimization , Policing, and Social Context (Clarendon Studies in Criminology) ---- SalesRank: 3049934
   ID 223783 Corporate Fraud ---- SalesRank: 2662778
   ID 78671 European Democracies Against Terrorism: Governmental Policies and Intergovernmental Cooperation (Oati International Series in Law and Society) ---- SalesRank: 2651679
   ID 25715 Money Laundering and Financial Intermediaries (Studies in Comparative Corporate and Financial Law, V. 8.) ---- SalesRank: 2422943
   ID 18797 Tradition of the Law and Law of the Tradition : Law, State, and Social Control in China (Contributions in Criminology and Penology) ---- SalesRank: 2411074
   ID 84905 Common Sense Police Supervision: A How-To Manual f

#5) Listar os 10 produtos com a maior média de ratings por grupo (Group) de produtos

In [ ]:
filtrado = lines.map(lambda x: (x['group'],x['avg_rating'])).groupByKey().map(lambda x: (x[0],sorted(list(x[1])))).map(lambda x: (x[0],x[1][-10:])).collect()

busca = lines.map(lambda x: (x['title'], x['group'], x['avg_rating']))
busca = busca.map(oi2).filter(lambda x: x['result'] != []).collect()

contador = {
    'Book':0,
    'Music':0,
    'Software':0,
    'Video':0,
    'CE':0,
    'Baby Product': 0,
    'DVD': 0,
    'Video Games':0,
    'Toy':0
}

vet = []

for i in busca:
  index = i['result'][0]['group']
  if index in list(contador.keys()) and contador[index] < 10:
    contador[index] += 1
    vet.append(i['result'][0])

vet = sorted(vet, key = lambda i: (i['group'], float(i['avg'])))
group = vet[0]['group']
print('Group:',group)

for i in vet:
  if i['group'] !=  group:
    group = i['group']
    print('Group:',group)
  
  print(i)

# for i in filtrado:
#   print(i[0],len(i[1]))

Group: Baby Product
{'title': "Baby'S Record Keeper And Memory Box", 'group': 'Baby Product', 'avg': '4.5'}
Group: Book
{'title': 'Patterns of Preaching: A Sermon Sampler', 'group': 'Book', 'avg': '5'}
{'title': 'World War II Allied Fighter Planes Trading Cards', 'group': 'Book', 'avg': '5'}
{'title': 'Resetting the Clock : Five Anti-Aging Hormones That Improve and Extend Life', 'group': 'Book', 'avg': '5'}
{'title': 'Computed Tomography : Fundamentals, System Technology, Image Quality, Applications', 'group': 'Book', 'avg': '5'}
{'title': 'Backroad Buffets & Country Cafes: A Southern Guide to Meat-And-Threes & Down-Home Dining', 'group': 'Book', 'avg': '5'}
{'title': 'Double Jeopardy (T*Witches, 6)', 'group': 'Book', 'avg': '5'}
{'title': 'Renovations: A Father and Son Rebuild a House and Rediscover Each Other', 'group': 'Book', 'avg': '5'}
{'title': 'How to Kill a Monster (Goosebumps)', 'group': 'Book', 'avg': '5'}
{'title': 'Baby Boomer Retirement: 65 Simple Ways to Protect Your Fut

#6) Listar a 5 categorias de produto com a maior média geral de ratings 

In [ ]:
from operator import add

def distribui(word):
  array = []
  #return word[1]
  for i in word:
    for j in i:
      if len(j) != 1 and float(word[1]) > 0:
        array.append((j,float(word[1])))
  return array

def myCountByKey(rdd):
    return rdd.map(lambda row: (row[0], 1)).reduceByKey(add)

def getMedia(word):
  return (word[0],round(word[1][0]/word[1][1],4))

filtrado = lines.map(lambda x: (x['categories'], x['avg_rating'])).map(distribui).flatMap(lambda x: x)

ocorrencia = myCountByKey(filtrado)

media = filtrado.reduceByKey(lambda x,y: x+y).join(ocorrencia).map(getMedia).sortBy(lambda x: -x[1]).take(5)

for i in media:
  print(i)




('Image Composer[4154]', 5.0)
('Religion & Spirituality[3201791]', 5.0)
('Lewis, Shari[151693]', 5.0)
('Step into Reading Phonics[12959581]', 5.0)
('Brother Lawrence[173026]', 5.0)


#7) Listar os 10 clientes que mais fizeram comentários por grupo (Group) de produto

In [ ]:
def oi(word):
  vet = []
  for i in word['reviews']:
    vet.append(((word['group'],i['cliente']), 1))

  return vet

filtrado = lines.filter(lambda x: x['reviews']!=[]).map(oi)

flat = filtrado.flatMap(lambda x: x).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],(x[0][1],x[1]))).sortBy(lambda x: (x[0],-x[1][1])).collect()


contador = 0
group = flat[0][0]

print('Group:',group)

for i in flat:

  if i[0] != group:
    group = i[0]
    contador = 0
    print('Group:',group)

  if contador <= 10:
      print(i[1])
      contador+=1


Group: Baby Product
('A37TFIP0OMKGMW', 1)
('AI9SB5VKUFXDC', 1)
('A2LAH8VX720175', 1)
Group: Book
('ATVPDKIKX0DER', 643185)
('A3UN6WX5RRO2AG', 154531)
('A14OJS0VWMOSWO', 9589)
('AFVQZQ8PW0L', 5441)
('A1K1JW1C5CUSUZ', 3562)
('A2NJO6YE954DBH', 2055)
('A3QVAKVRAH657N', 1651)
('A1NATT3PN24QWY', 1535)
('A1D2C0WDCSHUWZ', 1508)
('A2ODBHT4URXVXQ', 1469)
('A3SN9EF7GTNEU2', 1387)
Group: CE
('A1328SYT22GA4U', 1)
('A13JU90C7AU3RT', 1)
('A1J62O1S6QTHZJ', 1)
('A1SFX3CR838F36', 1)
('A2IX9TMXDBUCYV', 1)
Group: DVD
('ATVPDKIKX0DER', 63148)
('A3UN6WX5RRO2AG', 15549)
('A2NJO6YE954DBH', 1366)
('AU8552YCOO5QX', 1213)
('A3P1A63Q8L32C5', 859)
('A3LZGLA88K0LA0', 856)
('A82LIVYSX6WZ9', 683)
('A152C8GYY25HAH', 675)
('A16CZRQL23NOIW', 651)
('A1CZICCYP2M5PX', 650)
('AJKWF4W7QD4NS', 612)
Group: Music
('ATVPDKIKX0DER', 166149)
('A3UN6WX5RRO2AG', 15875)
('A9Q28YTLYREO7', 2760)
('A2U49LUUY4IKQQ', 1258)
('A1GN8UJIZLCA59', 1154)
('A2NJO6YE954DBH', 1128)
('A1J5KCZC8CMW9I', 1031)
('A3MOF5KF93Q6WE', 989)
('AXFI7TAWD6H6X', 